### Deliverable 1: Preprocessing the Data for a Neural Network

In [64]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df1 = pd.read_csv("Resources/charity_data.csv")
application_df1.head(1)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1


In [65]:
# application_df1.info()

In [66]:
application_df1["STATUS"].unique()

array([1, 0], dtype=int64)

In [67]:
# Keep only STATUS = 1, which I hope is "Active."
application_df1 = application_df1[application_df1['STATUS'] == 1]

In [68]:
application_df1["STATUS"].unique()

array([1], dtype=int64)

In [69]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df1.drop(columns=["EIN", "NAME", "STATUS"])
application_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34294 entries, 0 to 34298
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34294 non-null  object
 1   AFFILIATION             34294 non-null  object
 2   CLASSIFICATION          34294 non-null  object
 3   USE_CASE                34294 non-null  object
 4   ORGANIZATION            34294 non-null  object
 5   INCOME_AMT              34294 non-null  object
 6   SPECIAL_CONSIDERATIONS  34294 non-null  object
 7   ASK_AMT                 34294 non-null  int64 
 8   IS_SUCCESSFUL           34294 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 2.6+ MB


In [70]:
# Determine the number of unique values in each column.
application_df2_cat = application_df2.dtypes[application_df2.dtypes == "object"].index.tolist()
application_df2[application_df2_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [71]:
# Look at APPLICATION_TYPE value counts for binning
counts_apptype = application_df2.APPLICATION_TYPE.value_counts()
counts_apptype

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [72]:
# Visualize the value counts of APPLICATION_TYPE
# counts_apptype.plot.density()

In [73]:
# Determine which values to replace if counts are less than ...?
replace_apptype = list(counts_apptype[counts_apptype < 500].index)

# Replace in dataframe
for app in replace_apptype:
    application_df2.APPLICATION_TYPE = application_df2.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df2.APPLICATION_TYPE.value_counts()

T3       27032
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [74]:
# Look at CLASSIFICATION value counts for binning
counts_class = application_df2.CLASSIFICATION.value_counts()
counts_class

C1000    17323
C2000     6073
C1200     4837
C3000     1918
C2100     1882
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [75]:
# Visualize the value counts of CLASSIFICATION
# counts_class.plot.density()

In [76]:
# Determine which values to replace if counts are less than ..?
replace_class = list(counts_class[counts_class < 1880].index)

# Replace in dataframe
for cls in replace_class:
    application_df2.CLASSIFICATION = application_df2.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2.CLASSIFICATION.value_counts()

C1000    17323
C2000     6073
C1200     4837
Other     2261
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [77]:
# Generate our categorical variable lists
application_df2_cat = application_df2.dtypes[application_df2.dtypes == "object"].index.tolist()
application_df2[application_df2_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [78]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df2[application_df2_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_df2_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [79]:
# Merge one-hot encoded features and drop the originals
application_df3 = application_df2.merge(encode_df,left_index=True, right_index=True)
application_df4 = application_df3.drop(application_df2_cat,1)
application_df4.head()

C:\Users\LaStella\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [80]:
application_df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34289 entries, 0 to 34293
Data columns (total 43 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ASK_AMT                       34289 non-null  int64  
 1   IS_SUCCESSFUL                 34289 non-null  int64  
 2   APPLICATION_TYPE_Other        34289 non-null  float64
 3   APPLICATION_TYPE_T10          34289 non-null  float64
 4   APPLICATION_TYPE_T19          34289 non-null  float64
 5   APPLICATION_TYPE_T3           34289 non-null  float64
 6   APPLICATION_TYPE_T4           34289 non-null  float64
 7   APPLICATION_TYPE_T5           34289 non-null  float64
 8   APPLICATION_TYPE_T6           34289 non-null  float64
 9   APPLICATION_TYPE_T7           34289 non-null  float64
 10  APPLICATION_TYPE_T8           34289 non-null  float64
 11  AFFILIATION_CompanySponsored  34289 non-null  float64
 12  AFFILIATION_Family/Parent     34289 non-null  float64
 13  A

In [81]:
# Split our preprocessed data into our features and target arrays
y = application_df4["IS_SUCCESSFUL"].values
X = application_df4.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\LaStella\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
# y
# X

In [83]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [84]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
        units=hidden_nodes_layer1, 
        input_dim=number_input_features, 
        activation="relu")
        )

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, 
    activation="tanh")
        )

# Output layer
nn.add(tf.keras.layers.Dense(
    units=1, 
    activation="sigmoid")
      )

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                516       
                                                                 
 dense_4 (Dense)             (None, 6)                 78        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


# Compile the model
nn.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

# Train the model
fit_model = nn.fit(X_train,y_train,epochs=1)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [85]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [86]:
# Compile the model
nn.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    period=5,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    callbacks=[cp_callback]
)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6894 - accuracy: 0.5578
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6812 - accuracy: 0.5688
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6797 - accuracy: 0.5689
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6791 - accuracy: 0.5728
Epoch 5/100
801/804 [============================>.] - ETA: 0s - loss: 0.6787 - accuracy: 0.5737
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6787 - accuracy: 0.5737
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6784 - accuracy: 0.5747
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6782 - accuracy: 0.5736
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6779 - accuracy: 0.5744
Epoch 9/100
804/804 [==============================] -

804/804 [==============================] - 2s 2ms/step - loss: 0.6735 - accuracy: 0.5838
Epoch 64/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6737 - accuracy: 0.5828
Epoch 65/100
791/804 [============================>.] - ETA: 0s - loss: 0.6735 - accuracy: 0.5841
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6736 - accuracy: 0.5837
Epoch 66/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6735 - accuracy: 0.5833
Epoch 67/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6737 - accuracy: 0.5836
Epoch 68/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6735 - accuracy: 0.5835
Epoch 69/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6735 - accuracy: 0.5830
Epoch 70/100
794/804 [============================>.] - ETA: 0s - loss: 0.6734 - accuracy: 0.5845
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 

In [87]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_3.h5")